# LDA parameter exploration

One of the biggest challenges of LDA is knowing how to set the model parameters.   
That is:

(i) The number of topics the model should look for and return  
(ii) The Dirichlet priors used for the doc_topic and topic_word distributions

Whereas some techniques have been developed for the first, the literature rarely discusses how to determine the second other than offerring a few rules of thumb:  
(a) Sparse priors (< 1) should be used  
(b) sklean: defaults both priors to 1/K (where K = number of topics)   
(c) [Talley et al (2011; in supplementary information](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5361216/)): claim that standard topic_word prior = 0.01 and standard heuristic for setting doc_topic prior is (0.05(L/K)) where L = average document length and K = number of topics

# 0: Set up

In [ ]:
#data manipulation and organisation
import pandas as pd
import numpy as np

#topic modelling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
import gensim

#visualisations
import pyLDAvis
import pyLDAvis.sklearn
import seaborn as sns
import matplotlib.pyplot as plt

#other
import os, random, pkg_resources

In [ ]:
%matplotlib inline

# 1: Load data

In [ ]:
#check if have full dataset in 1 file and load, otherwise look for full dataset in 2 files and combine
if os.path.isfile('data_files/final_dataset_full.csv'):
    publications_data = pd.read_csv('data_files/final_dataset_full.csv')
else:
    publications_1 = pd.read_csv('data_files/final_dataset_1.csv')
    publications_2 = pd.read_csv('data_files/final_dataset_2.csv')
    publications_data = pd.concat([publications_1, publications_2])

#publications_data.head()

In [ ]:
#1: create vector representation of vocabulary
vectorizer = CountVectorizer(max_df=0.95, min_df=2)

#create document_term_matrix
dtm = vectorizer.fit_transform(publications_data['full_text'].values.astype('U'))

#save feature names
feature_names = vectorizer.get_feature_names()

# 2: Number of topics

<b>A: Perplexity</b> can be used to determine optimal number of topics. It is calculated by training an LDA model on a subset of the data and then testing the model on the remaining data. The messure is based on the log likehood of the withheld dataset given the trained model parameters.   
The smaller the perplexity, the better.  

<b>B: Topic coherence</b> refers to interpretability of LDA retrieved topics by humans - i.e., ease of labeling retrieved words as consistently belonging to the same theme. While it has traditionally relied on human judgment, more recently automated techniques have been developed that try to emulate human judgment of topic coherence.  
The higher the topic coherence score, the better.  
  
Both measures are commonly employed to establish the optimal number of topics to retrieve when training LDA on a dataset.    

NOTE: [Research has shown that](https://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf ) good perplexity does not always (rarely) correspond to good topic coherence (human interpretability of the topics). In fact they seem to be inversely correlated.     

## A: Perplexity

#### Split data into training and test sets

In [ ]:
train, test = train_test_split(dtm, test_size=0.33)

#### Obtain perplexity scores

In [ ]:
perplexity = []
topic_num = []
for i in range(5, 101, 5):
    topic_num.append(i)
    lda = LatentDirichletAllocation(n_components = i, learning_method = 'online', random_state = 0, doc_topic_prior = .01, topic_word_prior = .01, max_iter = 50) 
    lda.fit(train)
    perplex = lda.perplexity(test)
    perplexity.append(perplex)

#### Example results

In [ ]:
#results from a run through of above code: 
#random_state = 0, doc_topic_prior & topic_word_prior = .01, max_iter = 50, learning_method = 'online'
topic_num = [i for i in range(5, 101, 5)]
perplexity = [9749.26233006, 9661.37241773, 9287.38411205, 9160.92753102, 9041.17114357, 9043.50366576, 
              9163.71528423, 9121.43655305, 9073.84644206, 9012.69594335, 9002.59538173, 8974.08041532, 
              9033.37862115, 9055.90255748, 9006.76588174, 8991.8306582, 8981.71820475, 9073.62659523, 
              9027.59020386, 8987.11843688]

#### Plot

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
line = ax.plot(topic_num, perplexity, 'bo-')
plt.xlabel('Number of topics')
plt.ylabel("Perplexity")

## B: Topic coherence (using gensim)

for more information (summarized below) see: http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf  

gensim's coherence measures allow us to evaluate coherence of top N terms of each topic  
While there are a number of methods offered, they can all be summarized as using some score metric to evaluate all possible word-pairs of the N top identified words   
The coherence metric is then simply the sum (or average) of these scores

The choices of confirmation measures in gensim are:  

<b>c_uci</b> (=c_pmi): based on "pointwise mutual information" (PMI) - probablities are estimated based on word co-occurrence counts which are derived from Wikipedia (which acts as external reference corpus)  

<b>c_npmi</b>: as above but uses normalized PMI i.e. NPMI  

<b>u_mass</b> : similar to above but probability is evaluated based on frequency of words in the documents used to learn the topics (rather than using external corpus such as Wikipedia)  

<b>c_v</b>: uses 'context vectors' for every topic word w  
context vector = word_occurrence counts determined using all words located +/- 5 tokens around the word  
BUT instead of counts, it was found that performance was best when normalized PMI (NPMI) of each element and the word w was used to represent the vector  
-- the j-th element of the context vector v of word w<sub>i</sub> i.e. v<sub>ik</sub> = NPMI(w<sub>i</sub>w<sub>j</sub>)  
THEN vector similarities are calculated between context vectors of all word-pairs to determine similarity (e.g. using cosine similarity)  

<b>EXAMPLES:</b>  
if we had 4 words in a topic (game, sport, ball, team), we would compute:   

c_uci = 1/6 * (PMI(game,sport) + PMI(game,ball) + PMI(game,team) + PMI(sport,ball) + PMI(sport,team) + PMI(ball,team))   
  
u_mass = 1/6*(log(P(sport|game)) + log(P(ball|game)) + log(P(ball|sport)) + log(P(team|game)) + log(P(team|sport)) + log(P(team|ball))
  
c_v = 1/6*(cos(v_game, v_sport) + cos(v_game, v_ball) + cos(v_game,v_team) + cos(v_sport, v_ball) + cos(v_sport, v_team) + cos(v_ball, v_team))  

NOTE: c_v was found to correlate the most with human ratings of topic coherence(see link at the top)

#### Get texts into gensim format

In [ ]:
texts = [[word for word in document.split()] for document in text_data['text'].tolist()]
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

#### Function for extracting N top words

In [ ]:
def get_top_words(topic_term_dist, feature_names, n_top):

    top_words_list = []
    
    for idx, item in enumerate(topic_term_dist):
        top_words = [feature_names[i] for i in item.argsort()[:-n_top - 1:-1]]
        top_words_list += [top_words]
    
    return top_words_list

#### Calculate coherence metrics for models with different numbers of topics - using c_v method

In [ ]:
n_top_words = 15
n_topics = [15, 20, 25, 30, 35]

topic_coherence = {}

for i in n_topics:
    lda = LatentDirichletAllocation(n_components=i, max_iter=50,
                                    learning_method='online',
                                    random_state = 0,
                                    doc_topic_prior = .01, 
                                    topic_word_prior = .01) 
    lda.fit(dtm)
    
    topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, None]
    feature_names = vectorizer.get_feature_names()
    top_words_list = get_top_words(topic_term_dists, feature_names, n_top_words)
    
    #coherence_model = gensim.models.coherencemodel.CoherenceModel(topics=top_words_list, corpus=corpus, dictionary=dictionary, coherence="u_mass")
    coherence_model = gensim.models.coherencemodel.CoherenceModel(topics=top_words_list, texts = texts, dictionary=dictionary, coherence="c_v")
    
    topic_coherence[i] = [coherence_model.get_coherence()] + coherence_model.get_coherence_per_topic()

#### Example data

In [ ]:
#example results from running above code using the c_v method on 15, 20, 25, 30 and 35 topics
#values are coherence value for each individual topic, calculated for top 15 words

topic_coherence_15 = [0.49191374814991395, 0.60349254258716867, 0.56140543396626852, 0.54585603473493571, 0.43179460101650202, 0.55415859587966632, 0.48911178743666284, 0.73988761990141139, 0.42394451752803952, 0.39907628215639618, 0.36841358699360693, 0.47000117082752141, 0.42174310275540816, 0.54023031780770248, 0.51412778866084319]
topic_coherence_20 = [0.53709337240783639, 0.49182273043204711, 0.46217628409166583, 0.62247817112539705, 0.56043767456247906, 0.48110453971544076, 0.48809927013725229, 0.46327971271107715, 0.44542659906313453, 0.44275290773763726, 0.51442908584596514, 0.51645097675426899, 0.48891527664833612, 0.50835822863819946, 0.5161102783822008, 0.46619155061108131, 0.66995835376419144, 0.6398622357019691, 0.56349118781460072, 0.45848929581878856]
topic_coherence_25 = [0.58719391726891035, 0.53967334377510534, 0.61101731216901134, 0.560789365020403, 0.64826789204862179, 0.46628946222299411, 0.48809927013725224, 0.65326824832997532, 0.42097815366897468, 0.44285374198526006, 0.67977971677016258, 0.6233640094427364, 0.60135986870757041, 0.50851269757514006, 0.55898973576869337, 0.70290817456886134, 0.29297952150040868, 0.52421376949406417, 0.51775442951306849, 0.47580031153426694, 0.68312755412800885, 0.64595802105714872, 0.62160286115749319, 0.47725244163557007, 0.61725975182915382]
topic_coherence_30 = [0.47980920783540248, 0.53809504808862341, 0.57375957642167452, 0.47584563906878635, 0.64131327024028373, 0.43419517916602379, 0.51304700025217631, 0.63264945101981629, 0.5112621832985641, 0.4476720491892599, 0.58985068939324881, 0.62111673457485683, 0.50438139281577021, 0.54725187533672792, 0.5465947699142587, 0.60543225248823085, 0.4546838607537661, 0.5693290841312163, 0.63232647439031919, 0.48242894384170104, 0.29934416453666657, 0.59242012982016901, 0.64576940504163216, 0.65034536627263362, 0.6935745593240511, 0.63346010694270327, 0.49348946267510502, 0.43552189284754239, 0.96056267974514042, 0.77162894401385551]
topic_coherence_35 = [0.4965611537494527, 0.53769309760224371, 0.31792626535965202, 0.53748226261348619, 0.62900539207323425, 0.49831321744628254, 0.51304700025217653, 0.6326494510198164, 0.74194441133988254, 0.44731919902821549, 0.72916675222984795, 0.6150440057640788, 0.563989230815229, 0.56567375834760314, 0.58634438082349272, 0.78029534238914644, 0.6618406890187416, 0.60268553325948571, 0.62686861833336749, 0.42578700453310908, 0.75967600047298811, 0.96056267974514031, 0.66284670514527544, 0.5585251128400619, 0.64328105611280506, 0.59261816283726565, 0.46407846204736442, 0.4470287289025055, 0.61502096136659645, 0.54831182297667358, 0.4667215579262699, 0.58657818221732905, 0.73185665708231751, 0.7088730627032187, 0.64797413635947509]

#coherence_data = {15:topic_coherence_15, 20:topic_coherence_20, 25:topic_coherence_25, 30:topic_coherence_30, 35:topic_coherence_35}
coherence_data = [topic_coherence_15,topic_coherence_20, topic_coherence_25, topic_coherence_30, topic_coherence_35]

#### Plot

In [ ]:
fig, axn = plt.subplots(5, 1, figsize = (10, 5), sharex=True)
for i in range(len(coherence_data)):
    sns.boxplot(data=coherence_data[i],ax=axn.flat[i], orient='h').set_ylabel(str(len(coherence_data[i]))+" topics")

In [ ]:
fig, axn = plt.subplots(5, 1, figsize = (10, 5), sharex=True)
for i in range(len(coherence_data)):
    sns.distplot(coherence_data[i], ax=axn.flat[i], kde_kws={"label":str(len(coherence_data[i]))+" topics"})

# 3: Priors

## A: Herfindahl-Hirschman Index (HHI)

AIM: Explore the impact of selected priors on topic assignment  

HHI = commonly employed measure of 'concentration' (in markets)  

We used inverse HHI scores to evaluate how many topics on average a paper was assigned for different doc_topic and topic_word priors (the inverse of HHI is more intuitive than HHI in this context)    

#### Get data (doc_topic distributions) for different LDA models (varying priors)

In [ ]:
#get doc_topic_distribution for different values of alpha and eta (as set out in priors array)
n_topics = 25
priors = [1, .5, .1, .05, .01, .005, .001]
priors_eval = {}
iteration = 0

for i in priors:
    alpha = i
    
    for j in priors:
        iteration += 1
        eta = j
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,learning_method='online', random_state = 0,
                                doc_topic_prior = eta, topic_word_prior = alpha)
        lda.fit(dtm)
        
        doc_topic_dists = lda.transform(dtm)
        priors_eval[iteration] = [alpha, eta, doc_topic_dists]

#### Calculate HHI and inverse HHI for the different models

In [ ]:
#save information from code run above
data_dict = {}
index = 0

for key in priors_eval.keys():
    
    #retrieve info
    alpha = priors_eval[key][0]
    eta = priors_eval[key][1]
    doc_topic_dist = priors_eval[key][2]
    
    #calculatesquare of sums and sum of squares (=HHI)
    squares = np.square(doc_topic_dist)
    squared_sum = np.square(np.sum(doc_topic_dist, axis=1))
    sum_squares = np.sum(squares, axis=1)
    
    #calculate inverse HHI = the square of the sum/the sum of the squares
    docs = squared_sum/sum_squares
    
    HHI = sum_squares.flatten()
    HHI_inv = docs.flatten()
    num_docs = doc_topic_dists.shape[0]
    
    for i in range(num_docs):
        index += 1
        data_dict[index] = [alpha, eta, HHI[i], HHI_inv[i]]
        
data=pd.DataFrame.from_dict(data_dict, orient='index')
data = data.rename(columns={0:'alpha', 1:'eta', 2:'HHI', 3:'HHI_inv'})
data.to_csv("HHI.csv")

#### Example data

In [ ]:
#load csv with data saved from run-through of above script
data = pd.read_csv("HHI.csv")
topic_word_prior = data.loc[data['eta'] == .01]
doc_topic_prior = data.loc[data['alpha'] == .01]

#### Plot

In [ ]:
fig, ax =plt.subplots(1,2, sharey=True, figsize=(15,5))

sns.boxplot(x="alpha", y="HHI_inv", data=topic_word_prior, ax=ax[0]).set_title("Topic word prior")
sns.boxplot(x="eta", y="HHI_inv", data=doc_topic_prior, ax=ax[1]).set_title("Doc topic prior")